In [20]:
# === CHAMPIONS LEAGUE DAY — FULL STANDINGS PIPELINE ===
# Skills: MultiIndex slicing, crosstab, groupby-agg, ranking, tidy (long) data
import pandas as pd
import numpy as np

pd.set_option("display.width", 140)
pd.set_option("display.max_rows", 50)

# ------------------------------------------------------
# 0) Tiny synthetic dataset: two seasons, three stages
# ------------------------------------------------------
matches = pd.DataFrame({
    "match_id":  np.arange(1, 13),
    "season":    ["2021/22"]*6 + ["2022/23"]*6,
    "stage":     ["Group","Group","Group","Quarter","Quarter","Semi"]*2,
    "date": [
        "2021-09-14","2021-09-15","2021-09-21",
        "2021-04-06","2021-04-13","2021-04-27",
        "2022-09-14","2022-09-15","2022-09-21",
        "2022-04-06","2022-04-13","2022-04-27"
    ],
    "home_team": ["Real Madrid","Man City","Bayern","AC Milan","PSG","Chelsea",
                  "Real Madrid","Man City","Bayern","AC Milan","PSG","Chelsea"],
    "away_team": ["PSG","Chelsea","Liverpool","Real Madrid","Bayern","Man City",
                  "PSG","Chelsea","Liverpool","Real Madrid","Bayern","Man City"],
    "home_score":[2,3,1,0,2,1, 1,4,2,1,3,2],
    "away_score":[1,1,3,2,2,0, 2,1,1,1,1,3]
})

matches.head()

# Parse dates, keep bad strings as NaT instead of raising errors.
matches["date"] = pd.to_datetime(matches["date"], errors="coerce")
print("Raw matches: \n", matches.head(6))

base = matches.copy()

# ------------------------------------------------------
# 1) Build “long” per-team-per-match table (tidy format)
# ------------------------------------------------------
# We make two views:
#  - 'home' rows seen from home team perspective (gf = goals for, ga = goals against)
#  - 'away' rows seen from away team perspective
home = (base[["match_id","season","stage","date","home_team","away_team","home_score","away_score"]]
        .rename(columns={"home_team" : "team" , "away_team" : "opp_team", "home_score" : "gf", "away_score" : "ga"}))
away = (base[["match_id","season","stage","date","away_team","home_team","away_score","home_score"]]
        .rename(columns={"away_team":"team","home_team":"opp_team","away_score":"gf","home_score":"ga"}))

# pd.concat stacks DataFrames vertically (one under the other).
# ignore_index=True rebuilds a clean 0..N-1 index.
long =  (pd.concat([home,away] , ignore_index=True)
        .sort_values(["season","stage","team","date"])
        .reset_index(drop=True))
print("\nLong (per-team-per-match) — first 10 rows:\n", long.head(10))

# ------------------------------------------------------
# 2) Match result & points per team per match
# ------------------------------------------------------
# np.select: vectorized IF/ELSE. You pass a list of conditions and matching outputs.

long["result"] = np.select(
    [long["gf"] > long["ga"] , long["gf"] < long["ga"]],
    ["Win","Loss"],
    default="Draw"
)

# Points rule: Win=3, Draw=1, Loss=0
long["points"] = np.select(
    [long["result"] == "Win", long["result"] == "Loss"],
    [3,1],
    default=0
)

# Quick check
print("\nResults/points preview:\n", long[["season","stage","team","gf","ga","result","points"]].head(8))

# ------------------------------------------------------
# 3) MultiIndex slicing basics (read-only demo)
# ------------------------------------------------------

df_mi = long.set_index(["season","stage","team"]).sort_index()

# .loc with a tuple selects those levels. This returns all matches for that slice.
example_slice = df_mi.loc[("2021/22","Group")]

print("\nSlice — 2021/22 Group (all teams):\n", example_slice[["opp_team","gf","ga","result"]].head())

# ------------------------------------------------------
# 4) Crosstab: Wins/Draws/Losses counts per (season, stage, team)
# ------------------------------------------------------
# pd.crosstab(rows, columns) counts combinations. Here: counts of result categories.

wld = pd.crosstab(
    index=[long["season"],long["stage"],long["team"]],
    columns=long["result"]
)

wld = wld.reset_index().rename_axis(None, axis=1)
print("\nWins/Draws/Losses counts:\n", wld.head(10))

# ------------------------------------------------------
# 5) Standings table: aggregate metrics per (season, stage, team)
# ------------------------------------------------------
# .agg with named tuples: ("new_column", "how_to_aggregate").

standings = (long
             .groupby(["season","stage","team"])
             .agg(
                 matches = ("match_id" , "count"),
                 wins = ("result" , lambda s: (s=="Win").sum()),
                 draws = ("result", lambda s: (s=="Draw").sum()),
                 losses= ("result", lambda s: (s=="Loss").sum()),
                 gf = ("gf", "sum"),
                 ga = ("ga", "sum"),
                 points = ("points", "sum")
             ).reset_index())

standings["gd"] = standings["gf"] - standings["ga"]  # goal difference

print(standings)

# Ranking rule inside each (season, stage): points ↓, gd ↓, gf ↓
# sort_values + groupby.cumcount() gives 1-based rank with clear tiebreaks.

standings = (standings
             .sort_values(["season","stage","points","gd","gf"],
                          ascending=[True,True,False,False,False])
             .assign(rank=lambda d: d.groupby(["season","stage"]).cumcount()+1))

print("\nStandings with rank:\n", standings)

# Top-2 per group/stage (like tournament advancement)
top2_per_stage = (standings
                  .sort_values(["season","stage","rank"])
                  .groupby(["season","stage"], group_keys=False)
                  .head(2))

print("\nTop-2 per (season, stage):\n", top2_per_stage)


# ------------------------------------------------------
# 6) Extra analysis drills
# ------------------------------------------------------
# A) Highest goal difference overall
idx_gd = standings["gd"].idxmax()
best_gd = standings.loc[idx_gd, ["season","stage","team","gd","points"]]
print("\nBest goal difference overall:\n", best_gd.to_dict())

# B) Undefeated teams within stage (losses == 0)
undefeated = standings[standings["losses"] == 0].copy()
print("\nUndefeated teams (no losses) by stage:\n", undefeated[["season","stage","team","matches","wins","draws","losses","points"]])

# ------------------------------------------------------
# 7) Save artifacts
# ------------------------------------------------------
standings.to_csv("standings_ucl.csv", index=False)
wld.to_csv("wins_draws_losses_crosstab.csv", index=False)
print("\nSaved: standings_ucl.csv, wins_draws_losses_crosstab.csv")












Raw matches: 
    match_id   season    stage       date    home_team    away_team  home_score  away_score
0         1  2021/22    Group 2021-09-14  Real Madrid          PSG           2           1
1         2  2021/22    Group 2021-09-15     Man City      Chelsea           3           1
2         3  2021/22    Group 2021-09-21       Bayern    Liverpool           1           3
3         4  2021/22  Quarter 2021-04-06     AC Milan  Real Madrid           0           2
4         5  2021/22  Quarter 2021-04-13          PSG       Bayern           2           2
5         6  2021/22     Semi 2021-04-27      Chelsea     Man City           1           0

Long (per-team-per-match) — first 10 rows:
    match_id   season    stage       date         team     opp_team  gf  ga
0         3  2021/22    Group 2021-09-21       Bayern    Liverpool   1   3
1         2  2021/22    Group 2021-09-15      Chelsea     Man City   1   3
2         3  2021/22    Group 2021-09-21    Liverpool       Bayern   3   1
3  